In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Read in F1 Grand Prix data from https://www.kaggle.com/cjgdev/formula-1-race-data-19502017
races_csv = pd.read_csv("races.csv") 
results_csv = pd.read_csv("results.csv")
circuits_csv = pd.read_csv("circuits.csv")
drivers_csv = pd.read_csv("drivers.csv")

In [3]:
circuits_csv.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park


In [4]:
circuits_csv = circuits_csv.set_index('circuitId')

In [5]:
circuits_csv = circuits_csv.loc[circuits_csv['country'] == 'USA']
circuits_csv.head()

,circuitRef,name,location,country,lat,lng,alt,url
circuitId,,,,,,,,
19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...
33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.0750,NaN,http://en.wikipedia.org/wiki/Phoenix_street_ci...
37,detroit,Detroit Street Circuit,Detroit,USA,42.3298,-83.0401,NaN,http://en.wikipedia.org/wiki/Detroit_street_ci...
42,dallas,Fair Park,Dallas,USA,32.7774,-96.7587,NaN,http://en.wikipedia.org/wiki/Fair_Park
43,long_beach,Long Beach,California,USA,33.7651,-118.1890,NaN,"http://en.wikipedia.org/wiki/Long_Beach,_Calif..."


In [6]:
circuits_csv = circuits_csv.drop([37, 42, 43, 44, 72])
circuits_csv

,circuitRef,name,location,country,lat,lng,alt,url
circuitId,,,,,,,,
19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...
33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.0750,NaN,http://en.wikipedia.org/wiki/Phoenix_street_ci...
46,watkins_glen,Watkins Glen,New York State,USA,42.3369,-76.9272,NaN,http://en.wikipedia.org/wiki/Watkins_Glen_Inte...
60,riverside,Riverside International Raceway,California,USA,33.9370,-117.2730,NaN,http://en.wikipedia.org/wiki/Riverside_Interna...
63,sebring,Sebring International Raceway,Florida,USA,27.4547,-81.3483,NaN,http://en.wikipedia.org/wiki/Sebring_Raceway
69,americas,Circuit of the Americas,Austin,USA,30.1328,-97.6411,NaN,http://en.wikipedia.org/wiki/Circuit_of_the_Am...


In [7]:
circuits_csv = circuits_csv.reset_index()
circuits_csv

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...
1,33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.0750,NaN,http://en.wikipedia.org/wiki/Phoenix_street_ci...
2,46,watkins_glen,Watkins Glen,New York State,USA,42.3369,-76.9272,NaN,http://en.wikipedia.org/wiki/Watkins_Glen_Inte...
3,60,riverside,Riverside International Raceway,California,USA,33.9370,-117.2730,NaN,http://en.wikipedia.org/wiki/Riverside_Interna...
4,63,sebring,Sebring International Raceway,Florida,USA,27.4547,-81.3483,NaN,http://en.wikipedia.org/wiki/Sebring_Raceway
5,69,americas,Circuit of the Americas,Austin,USA,30.1328,-97.6411,NaN,http://en.wikipedia.org/wiki/Circuit_of_the_Am...


In [8]:
races_csv = races_csv.drop([43])
races_csv.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,3/29/2009,6:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,4/5/2009,9:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,4/19/2009,7:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,4/26/2009,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,5/10/2009,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [9]:
races_csv.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,3/29/2009,6:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,4/5/2009,9:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,4/19/2009,7:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,4/26/2009,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,5/10/2009,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [10]:
races_csv = races_csv.loc[races_csv['name'].isin(['United States Grand Prix', 'United States Grand Prix West'])]
races_csv.head()

,raceId,year,round,circuitId,name,date,time,url
41,42,2007,7,19,United States Grand Prix,6/17/2007,17:00:00,http://en.wikipedia.org/wiki/2007_United_State...
61,62,2006,10,19,United States Grand Prix,7/2/2006,14:00:00,http://en.wikipedia.org/wiki/2006_United_State...
78,79,2005,9,19,United States Grand Prix,6/19/2005,14:00:00,http://en.wikipedia.org/wiki/2005_United_State...
97,98,2004,9,19,United States Grand Prix,6/20/2004,NaN,http://en.wikipedia.org/wiki/2004_United_State...
121,122,2003,15,19,United States Grand Prix,9/28/2003,NaN,http://en.wikipedia.org/wiki/2003_United_State...


In [11]:
cir_race = pd.merge(circuits_csv, races_csv, on='circuitId')
cir_race

,circuitId,circuitRef,name_x,location,country,lat,lng,alt,url_x,raceId,year,round,name_y,date,time,url_y
0,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,42,2007,7,United States Grand Prix,6/17/2007,17:00:00,http://en.wikipedia.org/wiki/2007_United_State...
1,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,62,2006,10,United States Grand Prix,7/2/2006,14:00:00,http://en.wikipedia.org/wiki/2006_United_State...
2,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,79,2005,9,United States Grand Prix,6/19/2005,14:00:00,http://en.wikipedia.org/wiki/2005_United_State...
3,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,98,2004,9,United States Grand Prix,6/20/2004,NaN,http://en.wikipedia.org/wiki/2004_United_State...
4,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,122,2003,15,United States Grand Prix,9/28/2003,NaN,http://en.wikipedia.org/wiki/2003_United_State...
5,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,139,2002,16,United States Grand Prix,9/29/2002,NaN,http://en.wikipedia.org/wiki/2002_United_State...
6,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,156,2001,16,United States Grand Prix,9/30/2001,NaN,http://en.wikipedia.org/wiki/2001_United_State...
7,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,172,2000,15,United States Grand Prix,9/24/2000,NaN,http://en.wikipedia.org/wiki/2000_United_State...
8,33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.0750,NaN,http://en.wikipedia.org/wiki/Phoenix_street_ci...,305,1991,1,United States Grand Prix,3/10/1991,NaN,http://en.wikipedia.org/wiki/1991_United_State...
9,33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.0750,NaN,http://en.wikipedia.org/wiki/Phoenix_street_ci...,321,1990,1,United States Grand Prix,3/11/1990,NaN,http://en.wikipedia.org/wiki/1990_United_State...


In [12]:
drivers_csv.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/5/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [13]:
results_csv.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1


In [ ]:
overallFastestLapTime

In [14]:
dri_res = pd.merge(drivers_csv, results_csv, on='driverId')
dri_res.head()

,driverId,driverRef,number_x,code,forename,surname,dob,nationality,url,resultId,...,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,...,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,27,...,5,4.0,56,46.548,5525103.0,53.0,3.0,01:35.5,209.033,1
2,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,57,...,13,0.0,56,NaN,NaN,25.0,19.0,01:35.5,203.969,11
3,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,69,...,3,6.0,66,4.187,5903238.0,20.0,3.0,01:22.0,204.323,1
4,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,90,...,2,8.0,58,3.779,5213230.0,31.0,2.0,01:26.5,222.085,1


In [15]:
combined_df = pd.merge(cir_race, dri_res, on='raceId')
combined_df = combined_df.drop(columns=['circuitRef', 'lat', 'lng', 'alt', 'url_x', 'url_y', 'url', 'name_y', 'time_x', 'round', 'constructorId', 'number_y', 'grid', 'position', 'positionText', 'milliseconds',
       'fastestLap', 'statusId', 'rank', 'resultId', 'number_x', 'driverRef', 'code'])
combined_df

,circuitId,name_x,location,country,raceId,year,date,driverId,forename,surname,dob,nationality,positionOrder,points,laps,time_y,fastestLapTime,fastestLapSpeed
0,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,7/1/1985,British,1,10.0,73,31:10.0,01:13.2,206.101
1,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,2,Nick,Heidfeld,10/5/1977,German,18,0.0,56,NaN,01:13.4,205.562
2,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,3,Nico,Rosberg,27/06/1985,German,16,0.0,68,NaN,01:14.1,203.753
3,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,4,Fernando,Alonso,29/07/1981,Spanish,2,8.0,73,1.5,01:13.3,206.003
4,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,5,Heikki,Kovalainen,19/10/1981,Finnish,5,4.0,73,41.4,01:14.0,203.94
5,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,8,Kimi,RÌ_ikkÌ¦nen,17/10/1979,Finnish,4,5.0,73,15.4,01:13.1,206.397
6,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,11,Takuma,Sato,28/01/1977,Japanese,19,0.0,13,NaN,01:16.7,196.807
7,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,13,Felipe,Massa,25/04/1981,Brazilian,3,6.0,73,12.8,01:13.4,205.658
8,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,14,David,Coulthard,27/03/1971,British,20,0.0,0,NaN,NaN,NaN
9,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,15,Jarno,Trulli,13/07/1974,Italian,6,3.0,73,66.7,01:14.0,203.891


In [ ]:
2.5 / (01:13.  
      )

In [30]:
min_lap_time = combined_df\
    .dropna(how='any')\
    .groupby('date')['fastestLapTime']\
    .min()\
    .to_frame()\
    .rename(columns={'fastestLapTime': 'overallFastestLapTime'})

In [32]:
df2 = pd.merge(combined_df, min_lap_time, on='date')

In [34]:
df2.loc[df2['fastestLapTime'] == df2['overallFastestLapTime']] 

,circuitId,name_x,location,country,raceId,year,date,driverId,forename,surname,dob,nationality,positionOrder,points,laps,time_y,fastestLapTime,fastestLapSpeed,overallFastestLapTime
5,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,8,Kimi,RÌ_ikkÌ¦nen,17/10/1979,Finnish,4,5.0,73,15.4,01:13.1,206.397,01:13.1
38,19,Indianapolis Motor Speedway,Indianapolis,USA,62,2006,7/2/2006,30,Michael,Schumacher,3/1/1969,German,1,10.0,73,34:35.2,01:12.7,207.527,01:12.7
56,19,Indianapolis Motor Speedway,Indianapolis,USA,79,2005,6/19/2005,30,Michael,Schumacher,3/1/1969,German,1,10.0,73,29:43.2,01:11.5,211.074,01:11.5
74,19,Indianapolis Motor Speedway,Indianapolis,USA,98,2004,6/20/2004,22,Rubens,Barrichello,23/05/1972,Brazilian,2,8.0,73,2.95,01:10.4,214.366,01:10.4
76,19,Indianapolis Motor Speedway,Indianapolis,USA,98,2004,6/20/2004,30,Michael,Schumacher,3/1/1969,German,1,10.0,73,40:29.9,01:10.4,214.327,01:10.4
93,69,Circuit of the Americas,Austin,USA,878,2012,11/18/2012,20,Sebastian,Vettel,3/7/1987,German,2,18.0,56,0.675,01:39.3,199.772,01:39.3
116,69,Circuit of the Americas,Austin,USA,898,2013,11/17/2013,20,Sebastian,Vettel,3/7/1987,German,1,25.0,56,39:17.1,01:39.9,198.754,01:39.9
137,69,Circuit of the Americas,Austin,USA,916,2014,11/2/2014,20,Sebastian,Vettel,3/7/1987,German,7,6.0,56,+1:35.734,01:41.4,195.768,01:41.4
148,69,Circuit of the Americas,Austin,USA,942,2015,10/25/2015,1,Lewis,Hamilton,7/1/1985,British,1,25.0,56,50:52.7,01:40.7,197.014,01:40.7
149,69,Circuit of the Americas,Austin,USA,942,2015,10/25/2015,3,Nico,Rosberg,27/06/1985,German,2,18.0,56,2.85,01:40.7,197.154,01:40.7


In [16]:
# combined_df.columns

Index(['circuitId', 'name_x', 'location', 'country', 'raceId', 'year', 'date',
       'driverId', 'forename', 'surname', 'dob', 'nationality',
       'positionOrder', 'points', 'laps', 'time_y', 'fastestLapTime',
       'fastestLapSpeed'],
      dtype='object')